In [1]:
import requests
import json
from pathlib import Path
import os
import sys
import pandas as pd
from datetime import datetime
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))

import utils as util
import rw_request as rw_rq

In [2]:
clientCrt = "../certificates/rw.crt"
clientKey = "../certificates/rw.key"

apiBaseUrl = "https://docucamrw.hesotech.eu/DocuCam/CameraOne/api/v1"
channelInfoUrl = apiBaseUrl + "/Data/ChannelInfo" 
imageApiUrl = apiBaseUrl + "/Data/ImageAndMeasurements"  # measurements are averages during scan time (scan time 5-30min)

headers = {'content-type': 'application/json'}

In [3]:
requestData = {
 
    # SiteName takes in two different parameters: [CameraOne, CameraTwo]
    # SceneName takes in different parameteres depending on SiteName:  [CameraOne [HospitalView]], [CameraTwo [SmallStreetSlide, StreetSlide, SchoolView]]

    # If you want to return all availalbe scans and images, comment out the Coordinates line.
   
    'SiteName': 'CameraTwo',
    'SceneName': 'SmallFieldSlide',
    'TimeRange': {
        'Minimum': '2022-01-01T00:00:00.000Z',
        'Maximum': '2023-02-27T15:43:16.000Z'
    },
    #'Coordinates': [ { 'Layer': 0, 'Row': 1, 'Col': 0 }, { 'Layer': 0, 'Row': 1, 'Col': 1 } ]  # comment out if you want to return all images
}

In [4]:
channelResponse = requests.get(channelInfoUrl, headers=headers, cert=(clientCrt, clientKey), allow_redirects=True)

if (channelResponse.status_code != 200):
    print('Something went wrong!')
    exit(-1)

In [5]:
response = requests.post(
    imageApiUrl,
    headers=headers,
    cert=(clientCrt, clientKey),
    allow_redirects=True,
    data=json.dumps(requestData)
)

if (response.status_code != 200):
    print('Something went wrong!')
    exit(-1)


In [6]:
channelInfoData = channelResponse.json()
data = response.json()

In [7]:
# obtain measurement labels
skip_rate = 2
measurement_labels = []
for item in channelInfoData:
    for index, (key, value) in enumerate(item.items()):
        if index % skip_rate == 0:
            measurement_labels.append(value)

# map labels with measurements and drop redundant columns
df = pd.DataFrame(data)
count = 0
for i in measurement_labels:
    df[i] =df['Values'].apply(lambda x: x[count] )
    count = count + 1
df = df.drop(columns=['Values'])

# Put it into function 
We wrap the data request into a nice reusable function and store it under the src folder and cann simply call it in the next step.

In [8]:
# get all data from camera one
camera_one_data = util.return_data(site = "CameraOne", scene = "HospitalView", all_coordinates = True)

In [9]:
# get all datasets from camera two
camera_two_data_1 = util.return_data(site = "CameraTwo", scene = "SmallFieldSlide", all_coordinates = True)
camera_two_data_2 = util.return_data(site = "CameraTwo", scene = "StreetSlide", all_coordinates = True)
camera_two_data_3 = util.return_data(site = "CameraTwo", scene = "SchoolView", all_coordinates = True)

In [10]:
# merge all four datasets and sort by 
df_all = pd.concat([camera_one_data, camera_two_data_1, camera_two_data_2, camera_two_data_3], 
                   keys=["CameraOne_HospitalView", "CameraTwo_SmallFieldSlide", "CameraTwo_StreetSlide", "CameraTwo_SchoolView"])

In [11]:
# save data
df_all.to_csv(os.path.abspath(os.path.join('..', 'data'))+'/data_all_cameras.csv', index=False)

In [12]:
df_all

Timestamp  \
CameraOne_HospitalView 0    2023-01-18T15:47:13.68219+00:00   
                       1   2023-01-18T16:00:00.026914+00:00   
                       2   2023-01-19T09:00:00.464237+00:00   
                       3   2023-01-19T16:00:00.012271+00:00   
                       4   2023-01-20T09:00:00.005397+00:00   
...                                                     ...   
CameraTwo_SchoolView   69  2023-02-26T14:00:00.009312+00:00   
                       70  2023-02-27T06:00:00.010685+00:00   
                       71  2023-02-27T14:00:00.011936+00:00   
                       72  2023-02-28T05:59:59.996628+00:00   
                       73  2023-02-28T14:00:00.005732+00:00   

                                                                   ImageUrls  \
CameraOne_HospitalView 0   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
                       1   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
                       2   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
                       3   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
                       4   [https://docucamrw.hesotech.eu/DocuCam/CameraO...   
...                                                                      ...   
CameraTwo_SchoolView   69  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   
                       70  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   
                       71  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   
                       72  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   
                       73  [https://docucamrw.hesotech.eu/DocuCam/CameraT...   

                          ACTUAL_TEMPERATURE HUMIDITY ILLUMINATION  \
CameraOne_HospitalView 0                17.1     67.0       2799.0   
                       1                17.1     67.0  1324.099976   
                       2           19.799999     52.0     100000.0   
                       3                17.5     71.0        122.0   
                       4                18.4     70.0       5659.0   
...                                      ...      ...          ...   
CameraTwo_SchoolView   69          16.299999     77.0       2054.0   
                       70          16.299999     77.0       2054.0   
                       71          16.299999     77.0       2054.0   
                       72          16.299999     77.0       2054.0   
                       73          16.299999     77.0       2054.0   

                          RAIN_COUNTER SUNSHINEDURATION WIND_SPEED  \
CameraOne_HospitalView 0         736.0            218.0       13.2   
                       1         736.0            218.0       10.6   
                       2    751.400024            487.0  39.700001   
                       3    751.400024            850.0        9.5   
                       4    753.099976           1122.0  16.299999   
...                                ...              ...        ...   
CameraTwo_SchoolView   69   814.200012           5558.0        7.0   
                       70   814.200012           5558.0        7.0   
                       71   814.200012           5558.0        7.0   
                       72   814.200012           5558.0        7.0   
                       73   814.200012           5558.0        7.0   

                          WIND_SPEED_CALC Rain.Day.Counter Rain.Diff  \
CameraOne_HospitalView 0         3.666667              0.0       0.0   
                       1         2.944444              0.0       0.0   
                       2        11.027778              0.0       0.0   
                       3         2.638889              0.0       0.0   
                       4         4.527778              0.0       0.0   
...                                   ...              ...       ...   
CameraTwo_SchoolView   69        1.944444              5.3       0.0   
                       70        1.944444              5.3       0.0   
  

In [13]:
# make dataframe completely visible:
from pprint import pprint
from IPython.display import display, HTML

#display(HTML(df_all.to_html()))

# To-Do
* allow for coordinate input in query function
* allow to query all different datasets for each site and each scence and combine

In [15]:
camera_two_data_3 = rw_rq.return_data(site = "CameraTwo", scene = "SchoolView", all_coordinates = True)

In [19]:
camera_two_data_3.head(10)

,Timestamp,ImageUrls,ACTUAL_TEMPERATURE,HUMIDITY,ILLUMINATION,RAIN_COUNTER,SUNSHINEDURATION,WIND_SPEED,WIND_SPEED_CALC,Rain.Day.Counter,Rain.Diff,Sunshine.Day.Counter,Sunshine.Diff,Site_Scence
0,2023-01-24T11:00:00.007051+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,15.100000,75.0,6083.0,754.900024,3577.0,11.700000,3.250000,0.3,0.0,200,3.0,CameraTwo_SchoolView
1,2023-01-24T14:00:00.00166+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,17.200001,69.0,5973.0,754.900024,3757.0,13.000000,3.611111,0.3,0.0,380,3.0,CameraTwo_SchoolView
2,2023-01-25T06:00:00.002058+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,18.000000,60.0,5727.0,754.900024,3924.0,12.500000,3.472222,0.3,0.0,547,3.0,CameraTwo_SchoolView
3,2023-01-25T11:00:00.012383+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,20.299999,59.0,100000.0,754.900024,4222.0,18.200001,5.055555,0.0,0.0,298,3.0,CameraTwo_SchoolView
4,2023-01-25T14:00:00.001477+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,19.650001,55.5,6360.5,754.900024,4402.0,25.800000,7.166667,0.0,0.0,478,4.0,CameraTwo_SchoolView
5,2023-01-26T06:00:00.006909+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,18.799999,56.0,8270.0,754.900024,4552.0,14.900000,4.138889,0.0,0.0,628,2.0,CameraTwo_SchoolView
6,2023-01-26T11:00:00.010408+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,19.900000,57.0,8615.0,754.900024,4851.0,21.500000,5.972222,0.0,0.0,299,2.0,CameraTwo_SchoolView
7,2023-01-26T14:00:00.005225+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,20.900000,52.0,6021.0,754.900024,5031.0,14.700000,4.083333,0.0,0.0,479,1.0,CameraTwo_SchoolView
8,2023-01-27T06:00:00.013248+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,15.900000,80.0,7434.0,767.299988,5149.0,9.500000,2.638889,12.4,0.0,597,2.0,CameraTwo_SchoolView
9,2023-01-27T14:00:00.007361+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraT...,18.100000,70.0,4686.0,773.200012,5584.0,13.400000,3.722222,5.9,0.0,435,2.0,CameraTwo_SchoolView


In [17]:
len(camera_two_data_3.ImageUrls.iloc[0])

979

In [6]:
# let's set some coordinates:
coordinates_list = [{'Layer': 0, 'Row': 1, 'Col': 0}, {'Layer': 0, 'Row': 0, 'Col': 1}, {'Layer': 0, 'Row': 1, 'Col': 1}]

# save as json to load in function:
json_object = json.dumps(coordinates_list)
with open("../data/coordinates.json", "w") as outfile:
    outfile.write(json_object)

In [7]:
# only query coordinates from json
camera_two_data_3 = rw_rq.return_data(site = "CameraTwo", scene = "SchoolView", all_coordinates = False)

In [10]:
camera_two_data_3.ImageUrls.iloc[0]

['https://docucamrw.hesotech.eu/DocuCam/CameraTwo/Image/5/0/1/0/2023-01-24T11-00Z',
 'https://docucamrw.hesotech.eu/DocuCam/CameraTwo/Image/5/0/0/1/2023-01-24T11-00Z',
 'https://docucamrw.hesotech.eu/DocuCam/CameraTwo/Image/5/0/1/1/2023-01-24T11-00Z']